In [129]:
import pandas as pd
import json
from pandas import json_normalize
import numpy as np
from pandasql import sqldf


In [130]:
#Loading the json files
r = pd.read_json('C:\\Users\\Karthoz\\OneDrive\\Documents\\FETCH\\receipts\\receipts.json', lines=True)
u = pd.read_json('C:\\Users\\Karthoz\\OneDrive\\Documents\\FETCH\\users\\users.json', lines=True)
b = pd.read_json('C:\\Users\\Karthoz\\OneDrive\\Documents\\FETCH\\brands\\brands.json', lines=True)


In [131]:
#Creating copies to make changes/clean in the data
df1=r.copy()
df2=u.copy()
df3=b.copy()

In [55]:
print("receipts: ",df1.shape)
print("users: ",df2.shape)
print("brands: ",df3.shape)


receipts:  (1119, 15)
users:  (495, 7)
brands:  (1167, 8)


Cleaning the data to have in suitable format and removing nesting

In [132]:
#cleaning the data/ unnesting the columns
df1['id']=pd.json_normalize(df1['_id'])
df1['create_Date']=pd.json_normalize(df1['createDate'])
df1['date_Scanned']=pd.json_normalize(df1['dateScanned'])
df1['finished_Date']=pd.json_normalize(df1['finishedDate'])
df1['modify_Date']=pd.json_normalize(df1['modifyDate'])
df1['pointsAwarded_Date']=pd.json_normalize(df1['pointsAwardedDate'])
df1['purchase_Date']=pd.json_normalize(df1['purchaseDate'])

df1=df1.drop(['createDate','_id','dateScanned','finishedDate','pointsAwardedDate','purchaseDate','modifyDate'],axis=1)



In [133]:
df1.shape

(1119, 15)

In [134]:
rows = []

# Iterate through each row in the DataFrame
for index, row in df1.iterrows():
    # Get the rewardsReceiptItemList for the current row
    items_list = row['rewardsReceiptItemList']
   
    # If items_list is a list, create a row for each item
    if isinstance(items_list, list):
        for item in items_list:
            # Create a copy of the current row
            new_row = row.copy()
            # Update the rewardsReceiptItemList with the current item
            new_row['rewardsReceiptItemList'] = item
            # Append the new row to the list of rows
            rows.append(new_row)

# Create a new DataFrame from the list of rows
result_df = pd.DataFrame(rows)

# Display the size of resulting DataFrame, each row has each item
print(result_df.shape)


(6941, 15)


In [135]:
df_new = pd.json_normalize(result_df['rewardsReceiptItemList'])
df_new = df_new.rename(columns={'pointsEarned': 'pointsEarned_item'})


In [136]:
#Final receipts table
df_new.reset_index(inplace=True, drop=True)
result_df.reset_index(inplace=True, drop=True)
receipts = pd.concat([result_df,df_new],axis=1)
receipts= receipts.drop(['rewardsReceiptItemList'],axis=1)
# receipts.columns.values[2] = 'pointsEarned_receipt'


In [137]:
receipts['create_Date'] = pd.to_datetime(receipts['create_Date'],unit='ms')
receipts['date_Scanned'] = pd.to_datetime(receipts['date_Scanned'],unit='ms')
receipts['finished_Date'] = pd.to_datetime(receipts['finished_Date'],unit='ms')
receipts['modify_Date'] = pd.to_datetime(receipts['modify_Date'],unit='ms')
receipts['pointsAwarded_Date'] = pd.to_datetime(receipts['pointsAwarded_Date'],unit='ms')
receipts['purchase_Date'] = pd.to_datetime(receipts['purchase_Date'],unit='ms')
receipts['modify_Date'] = pd.to_datetime(receipts['modify_Date'],unit='ms')


In [138]:
rewards = receipts[['id','barcode', 'description',
       'finalPrice', 'itemPrice', 'needsFetchReview', 'partnerItemId',
       'preventTargetGapPoints', 'quantityPurchased', 'userFlaggedBarcode',
       'userFlaggedNewItem', 'userFlaggedPrice', 'userFlaggedQuantity',
       'needsFetchReviewReason', 'pointsNotAwardedReason', 'pointsPayerId',
       'rewardsGroup', 'rewardsProductPartnerId', 'userFlaggedDescription']]

In [139]:
rewards.head()

,id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,userFlaggedPrice,userFlaggedQuantity,needsFetchReviewReason,pointsNotAwardedReason,pointsPayerId,rewardsGroup,rewardsProductPartnerId,userFlaggedDescription
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,26.00,5.0,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,10.00,1.0,USER_FLAGGED,Action not allowed for user and CPG,5332f5fbe4b03c9a25efd0ba,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,26.00,3.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,28.00,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
receipts = receipts.drop(['barcode', 'description',
       'finalPrice', 'itemPrice', 'needsFetchReview', 'partnerItemId',
       'preventTargetGapPoints', 'quantityPurchased', 'userFlaggedBarcode',
       'userFlaggedNewItem', 'userFlaggedPrice', 'userFlaggedQuantity',
       'needsFetchReviewReason', 'pointsNotAwardedReason', 'pointsPayerId',
       'rewardsGroup', 'rewardsProductPartnerId', 'userFlaggedDescription'],axis=1)

In [141]:
receipts.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,id,create_Date,date_Scanned,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned_item,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,FINISHED,26.0,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,FINISHED,11.0,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,All-receipts receipt bonus,5.0,1.0,REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,All-receipts receipt bonus,5.0,4.0,FINISHED,28.0,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
df2['id']=pd.json_normalize(df2['_id'])
df2['created_Date']=pd.json_normalize(df2['createdDate'])
df2['last_Login']=pd.json_normalize(df2['lastLogin'])
df2=df2.drop(['createdDate','_id','lastLogin'],axis=1)

df2.head()
df2['created_Date']=pd.to_datetime(df2['created_Date'],unit='ms')
df2['last_Login']=pd.to_datetime(df2['last_Login'],unit='ms')
users=df2.copy()
users.head()


,active,role,signUpSource,state,id,created_Date,last_Login
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858


In [143]:
df3.shape

(1167, 8)

In [144]:
df3['id']=pd.json_normalize(df3['_id'])
df_cpg=df3.cpg.apply(pd.Series)
df_cpg['$ref'].unique()  #we can delete this column as it has only 2 values and probably both are same and its typo 
df3['cpg_id']=pd.json_normalize(df_cpg['$id'])
df3['cpg_ref']=df_cpg['$ref']

df3 = df3.drop(['_id','cpg'],axis=1)
# df3.head()
brands=df3.copy()
brands.head()

,barcode,category,categoryCode,name,topBrand,brandCode,id,cpg_id,cpg_ref
0,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs


Creating Final tables : Rewards , Receipts, Users, Brands

In [145]:
rewards['barcode'] = rewards['barcode'].fillna(0)
rewards['barcode'] = rewards['barcode'].astype(str)

brands['barcode']=brands['barcode'].astype(str)


In [146]:
print("receipts: ",receipts.shape)
print("users: ",users.shape)
print("brands: ",brands.shape)
print("rewards: ",rewards.shape)


receipts:  (6941, 30)
users:  (495, 7)
brands:  (1167, 9)
rewards:  (6941, 19)


Data Quality 

In [147]:
receipts.info()
brands.info()
users.info()
rewards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 30 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   bonusPointsEarned                   5978 non-null   float64       
 1   bonusPointsEarnedReason             5978 non-null   object        
 2   pointsEarned                        6251 non-null   float64       
 3   purchasedItemCount                  6892 non-null   float64       
 4   rewardsReceiptStatus                6941 non-null   object        
 5   totalSpent                          6941 non-null   float64       
 6   userId                              6941 non-null   object        
 7   id                                  6941 non-null   object        
 8   create_Date                         6941 non-null   datetime64[ns]
 9   date_Scanned                        6941 non-null   datetime64[ns]
 10  finished_Date           

In [148]:
print(receipts.describe(include='object'))
print(users.describe(include='object'))
print(brands.describe(include='object'))
print(rewards.describe(include='object'))

                                  bonusPointsEarnedReason  \
count                                                5978   
unique                                                  9   
top     Receipt number 1 completed, bonus point schedu...   
freq                                                 4605   

       rewardsReceiptStatus                    userId  \
count                  6941                      6941   
unique                    4                       246   
top                FINISHED  600f29a64329897eac239049   
freq                   5918                       459   

                              id originalMetaBriteBarcode  \
count                       6941                       71   
unique                       679                        6   
top     600f2fc80a720f0535000030                            
freq                         459                       47   

        originalMetaBriteDescription brandCode    competitorRewardsGroup  \
count                     

SQL Queries ( 2 examples)

In [149]:
import pandasql

In [45]:
#3 
query3 = """
SELECT
    rewardsReceiptStatus,
    COALESCE(AVG(totalSpent),0) AS average_spend
FROM
    receipts
where
rewardsReceiptStatus IN ('FINISHED', 'REJECTED') 
GROUP BY
    rewardsReceiptStatus
    
"""
#using FINISHED for ACCEPTED
result3 = pandasql.sqldf(query3, locals())
print(result3)

  rewardsReceiptStatus  average_spend
0             FINISHED    1244.793472
1             REJECTED      19.902500


In [46]:
#4
query4 = """
SELECT
    rewardsReceiptStatus,
    COALESCE(SUM(purchasedItemCount),0) AS total_items_purchased
FROM
    receipts
WHERE
    rewardsReceiptStatus IN ('FINISHED', 'REJECTED')
GROUP BY
    rewardsReceiptStatus
    """
#using FINISHED for ACCEPTED
result4 = pandasql.sqldf(query4, locals())
print(result4)

  rewardsReceiptStatus  total_items_purchased
0             FINISHED              1364998.0
1             REJECTED                  740.0


In [152]:
query6 = """ 
WITH MaxCreatedDate AS (
    SELECT MAX(created_Date) AS max_date
    FROM users
),
FilteredUsers AS (
    SELECT u.id, u.created_Date
    FROM users u, MaxCreatedDate m
    WHERE u.created_Date >= datetime(m.max_date, '-6 months')
)
SELECT 
    b.name AS brand_name,
    COUNT(ri.id) AS transaction_count
FROM 
    FilteredUsers u
JOIN 
    receipts r ON u.id = r.userId
JOIN 
    rewards ri ON r.id = ri.id
JOIN 
    brands b ON ri.barcode = b.barcode
GROUP BY 
    b.name
ORDER BY 
    transaction_count DESC
LIMIT 1;

"""


result6 = pandasql.sqldf(query6, locals())
print(result6)

  brand_name  transaction_count
0   Tostitos               5118
